In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os

# RNN architecture using keras and sklearn
# !pip install sklearn
# !pip install keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import keras
from keras.layers import Dense, LSTM, BatchNormalization, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.regularizers import l2

# bayes optimization
# !pip install bayesian-optimization
from bayes_opt import BayesianOptimization
from functools import partial

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../data/match_features.csv")

In [3]:
# to be removed when the data is cleaned
df.drop(columns=['Unnamed: 0', 'MP'], inplace=True)

In [4]:
df.shape

(1142, 60)

In [5]:
data = df
batch_size = 32
epoch = 50
input_dim = 59

In [6]:
# invert the input df
df = df[::-1]

In [7]:
# feed the lstm with all features starting from the open price
# still contains the high
input_features = data.iloc[::, :-1].values
input_data = input_features
labels = data.iloc[::, -1].values

input_data.shape

(1142, 59)

In [8]:
X = input_data[::]
y = labels[::]

In [9]:
# 20% for as test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

In [10]:
X_train.shape

(913, 59)

In [11]:
# reshape input data
X_train = np.reshape(X_train, (X_train.shape[0],  input_dim, 1))
X_test = np.reshape(X_test, (X_test.shape[0], input_dim, 1))

In [12]:
X_train.shape

(913, 59, 1)

In [13]:
def get_model(input_shape, dropout1_rate=0.2, dropout2_rate=0.2, lr=0.0001):
    #  reference https://publications.lib.chalmers.se/records/fulltext/250411/250411.pdf
    model = Sequential()
    model.add(LSTM(units = 256, return_sequences = True, input_shape = input_shape))
    model.add(Dropout(dropout1_rate))

    model.add(LSTM(units = 256))
    model.add(Dropout(dropout2_rate))

    model.add(Dense(units = 1))
    # model.summary()
    return model

    # model.compile(optimizer=Adam(lr=lr), loss='mean_squared_error', metrics=['accuracy'])
    # model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch, batch_size=batch_size)
    # model.save('./model_store/LSTM2.model')
    '''
    model = Sequential()
    model.add(Dense(units=128, input_shape=(input_dim,)))
    model.add(Dense(units=128, activation='relu', activity_regularizer=l2(0.01)))
    model.add(Dense(units=128, activation='relu', activity_regularizer=l2(0.01)))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.0001), loss='mean_squared_error', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch, batch_size=batch_size)
    '''

iter_count = 1

def fit_with(input_shape, verbose, dropout1_rate, dropout2_rate, lr):
    global iter_count
    
    # reference https://keras.io/api/callbacks/model_checkpoint/
    
    # create model
    model = get_model(input_shape, dropout1_rate, dropout2_rate, lr)
    
    # create optimizer using adam
    optimizer = Adam(learning_rate=lr)
    
    # compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    
    # setup checkpoints callback
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'./model_store/lstm_checkpoints/checkpoint_{iter_count}',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
    
    iter_count += 1
    
    # fit model
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=epoch, batch_size=batch_size, callbacks=[model_checkpoint_callback])
    
    # evaluate with val dataset
    score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(f"test loss is {score[0]}, test accuracy is {score[1]}")
    
    return score[1]

In [14]:
verbose = 1
input_shape = (input_dim, 1)
fit_with_partial = partial(fit_with, input_shape, verbose)

get_model(input_shape=(input_dim, 1))

# bayesian optimization
pbounds = {'dropout1_rate': (0.1, 0.5), 'dropout2_rate': (0.1, 0.5), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(
    f = fit_with_partial,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=10,)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | dropou... | dropou... |    lr     |
-------------------------------------------------------------
Train on 913 samples, validate on 229 samples
Epoch 1/50
913/913 [==============================] - 6s 7ms/step - loss: 0.3811 - accuracy: 0.5060 - val_loss: 0.2570 - val_accuracy: 0.5721
Epoch 2/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2685 - accuracy: 0.5597 - val_loss: 0.2447 - val_accuracy: 0.5502
Epoch 3/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2552 - accuracy: 0.5586 - val_loss: 0.2395 - val_accuracy: 0.5633
Epoch 4/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2528 - accuracy: 0.5696 - val_loss: 0.2406 - val_accuracy: 0.5590
Epoch 5/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2469 - accuracy: 0.5575 - val_loss: 0.2382 - val_accuracy: 0.6288
Epoch 6/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2459 - accuracy: 0.5926 - val_loss: 0.2334 -

Epoch 6/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2473 - accuracy: 0.5783 - val_loss: 0.2430 - val_accuracy: 0.5546
Epoch 7/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2432 - accuracy: 0.5619 - val_loss: 0.2383 - val_accuracy: 0.6070
Epoch 8/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2466 - accuracy: 0.5783 - val_loss: 0.2349 - val_accuracy: 0.6114
Epoch 9/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2435 - accuracy: 0.5904 - val_loss: 0.2432 - val_accuracy: 0.6157
Epoch 10/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2415 - accuracy: 0.5685 - val_loss: 0.2462 - val_accuracy: 0.5852
Epoch 11/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2445 - accuracy: 0.5750 - val_loss: 0.2365 - val_accuracy: 0.6157
Epoch 12/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2408 - accuracy: 0.5882 - val_loss: 0.2341 - val_accuracy: 0.6288
Epo

Epoch 12/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2424 - accuracy: 0.5936 - val_loss: 0.2299 - val_accuracy: 0.6245
Epoch 13/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2417 - accuracy: 0.5772 - val_loss: 0.2370 - val_accuracy: 0.6070
Epoch 14/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2438 - accuracy: 0.5991 - val_loss: 0.2254 - val_accuracy: 0.5983
Epoch 15/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2382 - accuracy: 0.6068 - val_loss: 0.2268 - val_accuracy: 0.6157
Epoch 16/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2432 - accuracy: 0.5706 - val_loss: 0.2327 - val_accuracy: 0.6157
Epoch 17/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2409 - accuracy: 0.5871 - val_loss: 0.2396 - val_accuracy: 0.5852
Epoch 18/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2468 - accuracy: 0.5531 - val_loss: 0.2350 - val_accuracy: 0.6201

Epoch 18/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2412 - accuracy: 0.5794 - val_loss: 0.2360 - val_accuracy: 0.6157
Epoch 19/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2378 - accuracy: 0.5761 - val_loss: 0.2374 - val_accuracy: 0.6026
Epoch 20/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2423 - accuracy: 0.5904 - val_loss: 0.2363 - val_accuracy: 0.6376
Epoch 21/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2482 - accuracy: 0.5509 - val_loss: 0.2434 - val_accuracy: 0.5764
Epoch 22/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2404 - accuracy: 0.5936 - val_loss: 0.2299 - val_accuracy: 0.6157
Epoch 23/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2406 - accuracy: 0.5860 - val_loss: 0.2312 - val_accuracy: 0.6070
Epoch 24/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2394 - accuracy: 0.6013 - val_loss: 0.2353 - val_accuracy: 0.6288

Epoch 24/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2441 - accuracy: 0.5761 - val_loss: 0.2310 - val_accuracy: 0.6201
Epoch 25/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2426 - accuracy: 0.5926 - val_loss: 0.2364 - val_accuracy: 0.6245
Epoch 26/50
913/913 [==============================] - 6s 6ms/step - loss: 0.2504 - accuracy: 0.5641 - val_loss: 0.2387 - val_accuracy: 0.6070
Epoch 27/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2421 - accuracy: 0.5838 - val_loss: 0.2357 - val_accuracy: 0.5895
Epoch 28/50
913/913 [==============================] - 6s 7ms/step - loss: 0.2424 - accuracy: 0.5827 - val_loss: 0.2321 - val_accuracy: 0.6288
Epoch 29/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2379 - accuracy: 0.6090 - val_loss: 0.2407 - val_accuracy: 0.5808
Epoch 30/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2461 - accuracy: 0.5827 - val_loss: 0.2310 - val_accuracy: 0.6201

...

Epoch 26/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2412 - accuracy: 0.5904 - val_loss: 0.2358 - val_accuracy: 0.6114
Epoch 27/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2398 - accuracy: 0.5915 - val_loss: 0.2394 - val_accuracy: 0.6070
Epoch 28/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2352 - accuracy: 0.6166 - val_loss: 0.2301 - val_accuracy: 0.5852
Epoch 29/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2405 - accuracy: 0.6002 - val_loss: 0.2286 - val_accuracy: 0.6070
Epoch 30/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2377 - accuracy: 0.5980 - val_loss: 0.2364 - val_accuracy: 0.6245
Epoch 31/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2379 - accuracy: 0.5980 - val_loss: 0.2379 - val_accuracy: 0.5983
Epoch 32/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2374 - accuracy: 0.5958 - val_loss: 0.2358 - val_accuracy: 0.5983

Epoch 32/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2381 - accuracy: 0.5904 - val_loss: 0.2377 - val_accuracy: 0.5895
Epoch 33/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2376 - accuracy: 0.6024 - val_loss: 0.2321 - val_accuracy: 0.6201
Epoch 34/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2390 - accuracy: 0.5969 - val_loss: 0.2343 - val_accuracy: 0.6288
Epoch 35/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2387 - accuracy: 0.5871 - val_loss: 0.2331 - val_accuracy: 0.6332
Epoch 36/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2394 - accuracy: 0.5805 - val_loss: 0.2341 - val_accuracy: 0.6245
Epoch 37/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2405 - accuracy: 0.5947 - val_loss: 0.2407 - val_accuracy: 0.5677
Epoch 38/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2398 - accuracy: 0.5904 - val_loss: 0.2327 - val_accuracy: 0.6332

Epoch 38/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2286 - accuracy: 0.6177 - val_loss: 0.2288 - val_accuracy: 0.6070
Epoch 39/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2295 - accuracy: 0.6265 - val_loss: 0.2319 - val_accuracy: 0.5983
Epoch 40/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2266 - accuracy: 0.6342 - val_loss: 0.2290 - val_accuracy: 0.5852
Epoch 41/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2335 - accuracy: 0.6101 - val_loss: 0.2295 - val_accuracy: 0.6376
Epoch 42/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2300 - accuracy: 0.6188 - val_loss: 0.2319 - val_accuracy: 0.6201
Epoch 43/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2260 - accuracy: 0.6484 - val_loss: 0.2319 - val_accuracy: 0.6114
Epoch 44/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2255 - accuracy: 0.6353 - val_loss: 0.2258 - val_accuracy: 0.6288

Epoch 44/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2365 - accuracy: 0.6057 - val_loss: 0.2345 - val_accuracy: 0.6376
Epoch 45/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2412 - accuracy: 0.5761 - val_loss: 0.2338 - val_accuracy: 0.6332
Epoch 46/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2405 - accuracy: 0.5936 - val_loss: 0.2384 - val_accuracy: 0.5764
Epoch 47/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2391 - accuracy: 0.6134 - val_loss: 0.2345 - val_accuracy: 0.6201
Epoch 48/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2366 - accuracy: 0.5871 - val_loss: 0.2363 - val_accuracy: 0.6245
Epoch 49/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2375 - accuracy: 0.6002 - val_loss: 0.2386 - val_accuracy: 0.5895
Epoch 50/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2397 - accuracy: 0.5783 - val_loss: 0.2341 - val_accuracy: 0.6157

Epoch 50/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2366 - accuracy: 0.5958 - val_loss: 0.2343 - val_accuracy: 0.6157
test loss is 0.2343067024472499, test accuracy is 0.6157205104827881
|  19       |  0.6157   |  0.1      |  0.3509   |  0.0001   |
Train on 913 samples, validate on 229 samples
Epoch 1/50
913/913 [==============================] - 6s 7ms/step - loss: 0.3819 - accuracy: 0.5093 - val_loss: 0.2568 - val_accuracy: 0.5415
Epoch 2/50
913/913 [==============================] - 6s 6ms/step - loss: 0.2754 - accuracy: 0.5433 - val_loss: 0.2432 - val_accuracy: 0.5502
Epoch 3/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2531 - accuracy: 0.5761 - val_loss: 0.2420 - val_accuracy: 0.5721
Epoch 4/50
913/913 [==============================] - 5s 6ms/step - loss: 0.2500 - accuracy: 0.5674 - val_loss: 0.2371 - val_accuracy: 0.6114
Epoch 5/50
913/913 [==============================] - 5s 5ms/step - loss: 0.2462 - accuracy: 0.5849 - val_loss: 